In [1]:
# http://web.stanford.edu/~boyd/papers/pdf/admm_slides.pdf

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator, RegressorMixin

In [3]:
# Loads data.
boston = load_boston()
print(boston.data.shape)

(506, 13)


In [10]:
# Executes preprocessing of data.
mean = np.mean(boston.data, axis=0)
std = np.std(boston.data, axis=0)
A = (boston.data - mean)/std
b = boston.target
N = A.shape[0]

In [16]:
class Admm(BaseEstimator, RegressorMixin):
    def __init__(self, lambd=1.0, rho=1.0, max_iteration_count=1000):
        self.lambd = lambd
        self.rho = rho
        self.threshold = lambd / rho
        self.max_iteration_count = max_iteration_count
        self.coef_ = None
        self.intercept_ = 0.0

    def _soft_threshold(self, x):
        t = self.threshold
        
        positive_indexes = x >= t
        negative_indexes = x <= t
        zero_indexes = abs(x) <= t
        
        y = np.zeros(x.shape)    
        y[positive_indexes] = x[positive_indexes] - t
        y[negative_indexes] = x[negative_indexes] + t
        y[zero_indexes] = 0.0
    
        return y

    def fit(self, A, b):
        inv_matrix = np.linalg.inv(np.dot(A.T, A) + self.rho * np.identity(A.shape[1]))

        # # scikit-learn's solution
        #x = np.array([0, 0, 0, 0, 0, 2.71310728, 0, 0, 0, 0, -1.34349862, 0.18079388, -3.54361166])
        
        x = np.dot(A.T, b)
        z = x.copy()
        y = np.zeros(len(x))
    
        for iteration in range(self.max_iteration_count):
            x = np.dot(inv_matrix, np.dot(A.T, b) + self.rho * z - y)
            z = self._soft_threshold(x + y / self.rho)
            y += self.rho * (x - z)

        self.coef_ = x
                      
        return self

    def predict(self, X):
        y = np.dot(X, self.coef_)
        return y

In [17]:
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
model = Admm(lambd=N*1.06, rho=N*0.95, max_iteration_count=1000)
model.fit(A, b)
print("Lasso by ADMM")
print(model.coef_)

Lasso by ADMM
[ 0.0000  0.0000  0.0000 -0.0000  0.0000  2.6760  0.0000  0.0000  0.0000
  0.0000 -1.3074  0.1344 -3.5368]


In [21]:
from sklearn.model_selection import GridSearchCV
estimator = Admm(lambd=1.0, rho=1.0, max_iteration_count=1000)
gs = GridSearchCV(estimator, {'lambd': [N*1.06], 'rho': [N*0.95]})
gs.fit(A,b)
gs.best_estimator_, gs.best_params_, gs.best_score_

(Admm(lambd=536.36, max_iteration_count=1000, rho=480.7),
 {'lambd': 536.36, 'rho': 480.7},
 -32.895232108644777)

In [22]:
from sklearn import linear_model

model = linear_model.Lasso(alpha=1.0, max_iter=1000, tol=0.0, fit_intercept=False) # tol=0.0で収束判定なし(上の実装とほぼ同条件?)
model.fit(A, b)
print("")
print("scikit learn Lasso")
print(model.intercept_)
print(model.coef_)


scikit learn Lasso
0.0
[-0.0000  0.0000 -0.0000  0.0000 -0.0000  2.7131 -0.0000 -0.0000 -0.0000
 -0.0000 -1.3435  0.1808 -3.5436]
